In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from cityscape import CityscapeConfig, CityscapeDataset
from models import ResNet, FlowNet, MaskRCNN, Warp, Decision
from mrcnn import utils
import mrcnn.model as modellib

os.environ["CUDA_VISIBLE_DEVICES"] = '3'

Using TensorFlow backend.


In [ ]:
data_dir = '/data/cityscapes_dataset/cityscape'
config = CityscapeConfig()
config.IMAGE_SHAPE = [1024, 1024, 6]
config.Flow =True
config.POST_NMS_ROIS_INFERENCE = 500
#config.display()

# Validation dataset
offset = 20
dataset_val = CityscapeDataset()
dataset_val.load_cityscape(data_dir, "val", offset)
dataset_val.prepare()


print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))


In [3]:
resnet = ResNet(config=config)
flownet = FlowNet(config=config)
maskrcnn = MaskRCNN(config=config)
warp = Warp(config=config)
decision = Decision()

model_path = "/home/susean/Mask_RCNN/logs/mask_rcnn_cityscapes_0040.h5"
resnet.load_weights(model_path, by_name=True)
flownet.load_weights(model_path, by_name=True)
maskrcnn.load_weights(model_path, by_name=True)
decision.load_weights("/home/susean/Mask_RCNN/logs/deciosion_0181_2.59.h5", by_name=True)

In [ ]:
APs = []
threshold = 85
seg_step = 0
flow_step = 0
score = 0
for image_id in range(len(dataset_val.image_ids)):
     # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config,
                               image_id, use_mini_mask=False)

    current = np.expand_dims(image[:,:,3:], 0)
    image_metas = np.expand_dims(image_meta, 0)
    
    #start_time = time.time()
    if image_id % offset != 0:
        images = np.concatenate([current, key], 3)
        flow, flow_feature = flownet.keras_model.predict(images)
        score = decision.keras_model.predict(flow_feature)

    if score < threshold or image_id % offset == 0:
        seg_step += 1
        key_P2, key_P3, key_P4, key_P5, key_P6 = resnet.keras_model.predict(current)

        key = current
        P2, P3, P4, P5, P6 = key_P2, key_P3, key_P4, key_P5, key_P6
    else:
        flow_step += 1
        P2, P3, P4, P5, P6 = warp.predict([key_P2, key_P3, key_P4, key_P5, key_P6, flow])

    inputs=[image_metas, P2, P3, P4, P5, P6]
    result = maskrcnn.detect_molded(inputs)
    #data_time = time.time() - start_time
    #print("times {:.6f}".format(data_time))
    #print(score)
    
    if np.sum(result["scores"]) == 0:
        print("{} Fasle".format(image_id))
        continue

    # Compute AP
    if (image_id+1) % offset == 0:
        AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                        result["rois"], result["class_ids"], result["scores"], result['masks'])
        #AP = utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
        #               result["rois"], result["class_ids"], result["scores"], result['masks'], verbose=0)
        APs.append(AP)
        print("step: {:3d}, mAP: {:.3f}".format(image_id, np.mean(APs)))
        
print("step: {:3d}, mAP: {:.3f}".format(image_id, np.mean(APs)))
print("segmentation steps:", seg_step, "flow steps:", flow_step)